In [ ]:
df=read.csv("C:/Users/student/Downloads/R/adult.csv", header=FALSE)
str(df)
names(df)=c('age','workclass','fnlwt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','income')
str(df)

missvar= function(data){
  for (i in 1:ncol(data)){
  print(c(names(data[i]), sum(is.na(data[,i]))))
  } 
}
missvar(df)

df[df==" ?"]=NA
sum(is.na(df))

misshandled=function(data){
  for(i in 1:ncol(data)){
    if (is.numeric(data[,i])){
      data[,i][is.na(data[,i])]=mean(data[,i], na.rm=TRUE)
    } else{
      levels=unique(data[,i])
      data[,i][is.na(data[,i])]=levels[which.max(tabulate(match(data[,i],levels)))]
    }
  }
  return(data)
}
newdf=misshandled(df)
sum(is.na(df))
sum(is.na(newdf))
```
###6.	Perform data visualization on the data.  You should include, at least:
#-	Bar charts of two categorical variables
#-	Density curves of numeric variables on the categories of categorical variables.  For example: plotting density curves of Age on female and male. 

library(ggplot2)
bar_var=function(data, i, j){
  if(is.numeric(data[,i])| is.numeric(data[,j])){
    print("At least one of the columns is not categorical")
  } else{
    print(ggplot(data)+geom_bar(mapping=aes(x=data[,i], fill=data[,j]), position="dodge")+labs(x=names(data)[i], fill=names(data)[j]))
  }
}
bar_var(newdf,10,9)
bar_var(newdf,10,15)

density_var=function(data,i,j){
  if(!is.numeric(data[,i])){
    print("The input variable is not numeric")
  } else if(is.numeric(data[,j])){
    print("The partitioning variable is not categorical")
  } else{
    print(ggplot(data)+geom_density(mapping=aes(x=data[,i], color=data[,j]))+labs(x=names(data)[i], color=names(data)[j]))
  }
}
density_var(newdf,1,9)
density_var(newdf,1,15)

library(caret)
splitIndex<- createDataPartition(df$income, p=.6, list=FALSE, times = 1)
train<- df[splitIndex,]
test<- df[-splitIndex,]
#test 60:40
print(nrow(train)/nrow(test)) 

library(rpart)
mytree <- rpart(income ~ ., data = train, method = "class")
pred <- predict(mytree,test, type = "class")
head(pred)

cm=confusionMatrix(data = pred, reference = test$income, positive=" >50K")
cm

library(rattle)
fancyRpartPlot(mytree)

treemodel=function(data,j){
  library(caret)
  splitIndex<- createDataPartition(df$income, p=j, list=FALSE, times = 1)
  train<- data[splitIndex,]
  test<- data[-splitIndex,]
  library(rpart)
  mytree <- rpart(income~ ., data = train, method = "class")
  pred <- predict(mytree,test, type = "class")
  cm=confusionMatrix(data = pred, reference = test$income, positive=" >50K")
  library(rattle)
  print(j)
  print(cm)
  fancyRpartPlot(mytree)
}
treemodel(newdf,.7)
treemodel(newdf,.8)
treemodel(newdf,.9)